In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import numpy as np
from brukeropusreader import read_file
from scipy.interpolate import interp1d
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [ ]:



# Define folder paths (corrected paths for RE and OE)
folder_path_oe = 'Spectra RE'  # OE data actually in 'Spectra RE'
folder_path_re = 'Spectra OE'  # RE data actually in 'Spectra OE'
folder_path_trachea = 'Spectra TR'  # Trachea data is correct

# Function to list files in a folder
def list_files(folder_path):
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    return files

# List files in each folder
files_re = list_files(folder_path_re)  # Actually RE data in 'Spectra OE'
files_oe = list_files(folder_path_oe)  # Actually OE data in 'Spectra RE'
files_tr = list_files(folder_path_trachea)  # Trachea data in 'Spectra TR'

# Displaying file names and their count
print("RE Folder:")
print(f"Number of files: {len(files_re)}")
print("Files:", files_re)

print("\nOE Folder:")
print(f"Number of files: {len(files_oe)}")
print("Files:", files_oe)

print("\nTrachea Folder:")
print(f"Number of files: {len(files_tr)}")
print("Files:", files_tr)




In [ ]:
# Define the common set of wavenumbers
start_wavenumber = 5000  # Starting wavenumber
end_wavenumber = 400  # Ending wavenumber
resolution = 1  # Resolution in cm⁻¹

# Generate the common set of wavenumbers
common_wavenumbers = np.arange(start_wavenumber, end_wavenumber - resolution, -resolution)

# Verify the first and last values as well as the size of the set to ensure it matches expectations
print("First wavenumber:", common_wavenumbers[0])
print("Last wavenumber:", common_wavenumbers[-1])
print("Total number of points:", len(common_wavenumbers))

In [ ]:
def get_wavenumbers(file_path):
    opus_data = read_file(file_path)
    # Assuming wavenumbers are stored in a key 'WN' in the returned data
    num_points = len(opus_data['AB'])
    start_wn = opus_data['AB Data Parameter']['FXV']
    end_wn = opus_data['AB Data Parameter']['LXV']
    # Generate the list of wavenumbers
    wavenumbers = np.linspace(start_wn, end_wn, num_points)
    return wavenumbers

In [ ]:
def get_spectral_data(file_path):
    """
    Extracts spectral data (e.g., absorbance values) from a file.

    Parameters:
    - file_path: The path to the Bruker file.

    Returns:
    - A numpy array containing the spectral data.
    """
    opus_data = read_file(file_path)
    # Assuming the spectral data is stored in the 'AB' key
    spectral_data = opus_data['AB']
    return spectral_data

In [ ]:
def interpolate_to_common_wavenumbers(file_path, common_wavenumbers):
    """
    Interpolates the spectral data from a file to a common set of wavenumbers.
    
    Parameters:
    - file_path: The path to the file.
    - common_wavenumbers: The common wavenumbers to interpolate the data to.
    
    Returns:
    - A numpy array of the interpolated spectral data.
    """
    original_wavenumbers = get_wavenumbers(file_path)
    original_data = get_spectral_data(file_path)
    interpolation_func = interp1d(original_wavenumbers, original_data, kind='linear', fill_value="extrapolate")
    interpolated_data = interpolation_func(common_wavenumbers)
    return interpolated_data

In [ ]:
def z_score_normalize(data):
    """
    Normalizes a data array using Z-score normalization.
    
    Parameters:
    - data: A numpy array of spectral data.
    
    Returns:
    - The Z-score normalized spectral data.
    """
    mean = np.mean(data)
    std_dev = np.std(data)
    normalized_data = (data - mean) / std_dev
    return normalized_data

In [ ]:
def process_directory(folder_path, common_wavenumbers):
    """
    Processes all files in a directory, interpolating and normalizing their spectral data.
    
    Parameters:
    - folder_path: The path to the directory.
    - common_wavenumbers: The common wavenumbers to interpolate the data to.
    
    Returns:
    - A numpy array of all the normalized spectral data from the directory.
    """
    normalized_data_list = []
    files = list_files(folder_path)
    for file in files:
        file_path = os.path.join(folder_path, file)
        data = interpolate_to_common_wavenumbers(file_path, common_wavenumbers)
        
        # Cut the spectra to the wavenumbers from 800 to 1900 cm⁻¹
        mask = (common_wavenumbers >= 900) & (common_wavenumbers <= 1800)
        data = data[mask]
        
        normalized_data = z_score_normalize(data)
        normalized_data_list.append(normalized_data)
    return np.array(normalized_data_list)


In [ ]:
# Verify the first and last values as well as the size of the set to ensure it matches expectations
print("First wavenumber:", common_wavenumbers[0])
print("Last wavenumber:", common_wavenumbers[-1])
print("Total number of points:", len(common_wavenumbers))


In [ ]:
# Process directories
data_oe = process_directory(folder_path_oe, common_wavenumbers)
data_re = process_directory(folder_path_re, common_wavenumbers)
data_trachea = process_directory(folder_path_trachea, common_wavenumbers)


In [ ]:
# Split the data into a temporary training dataset and final test set
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the temporary training dataset into actual training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Define the parameter space for Bayesian Optimization
param_space = {
    'C': Real(1e-6, 1e+6, prior='log-uniform'),
    'gamma': Real(1e-6, 1e+1, prior='log-uniform'),
    'kernel': Categorical(['linear', 'poly', 'rbf', 'sigmoid']),
    'degree': Integer(1, 8),  # Only relevant for 'poly' kernel
    'coef0': Real(0, 10)  # Only relevant for 'poly' and 'sigmoid' kernels
}

# Initialize the SVC model
model = SVC(probability=True, random_state=42)

# Initialize the Bayesian search with cross-validation
bayes_search = BayesSearchCV(estimator=model,
                             search_spaces=param_space,
                             n_iter=32,  # Number of iterations of the search
                             cv=5,  # 5-fold cross-validation
                             n_jobs=-1,  # Use all available CPUs
                             random_state=42)

# Fit the model using Bayesian optimization
bayes_search.fit(X_train, y_train)

# Define class names
class_names = ['OE', 'RE', 'Trachea']
               
# Get the best parameters found by Bayesian optimization
best_params = bayes_search.best_params_
print("Best parameters found by Bayesian Optimization:", best_params)

# Evaluate the optimized model
y_pred = bayes_search.predict(X_test)
print(classification_report(y_test, y_pred, target_names=class_names))
print(confusion_matrix(y_test, y_pred))



In [ ]:
# Führe 5-fache Cross-Validation nur auf dem Trainingsdatensatz durch
scores = cross_val_score(model, X_train, y_train, cv=5)

# Berechne den Durchschnitt und die Standardabweichung der Scores
average_score = np.mean(scores)
std_deviation = np.std(scores)

# Gib die Ergebnisse aus, auf 2 Dezimalstellen gerundet
print("Cross-validation scores (on training set):", scores)
print("Average score:", f"{average_score:.2f}")
print("Standard deviation of scores:", f"{std_deviation:.2f}")


In [ ]:
# Compute and plot ROC curves
# Get the predicted probabilities for each class
y_prob = model.predict_proba(X_test)

# Compute ROC curve and ROC area for each class
plt.figure(figsize=(8, 6))
for class_index in range(len(class_names)):
    fpr, tpr, _ = roc_curve(y_test == class_index, y_prob[:, class_index])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'ROC curve for {class_names[class_index]} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
# Function to plot learning curve
def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10)):
    plt.figure()
    plt.title(title)
    plt.xlabel("Training examples")
    plt.ylabel("Score")

    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

# Initialize the Decision Tree model
model = DecisionTreeClassifier(random_state=42)

# Plot learning curve for the Decision Tree model
title = "Learning Curves (Decision Tree)"
cv = 10  # Number of folds in cross-validation
plot_learning_curve(model, title, X, y, cv=cv)
plt.show()